### Все необходимые пипы и импорты

In [1]:
from pymystem3 import Mystem
m = Mystem()
import string
import gensim
import gensim.corpora as corpora
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

In [2]:
from typing import List
import json
import os
from tqdm.auto import tqdm
import nltk
import pandas as pd
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
from nltk.collocations import BigramAssocMeasures, TrigramAssocMeasures, BigramCollocationFinder, TrigramCollocationFinder
import re
import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: ignored

In [3]:
!pip install deplacy natasha
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

### Эксперименты с биграммами и триграммами

##### **Эксперимент 1**

In [6]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

Корпус: 10 документов – по одному на каждого автора. 

In [7]:
corpus = []
all_texts = soviet + emig
for a in tqdm(all_texts):
    with open(f'{a}_леммы.txt', encoding='utf-8') as f:
        text = f.read()
        corpus.append(text)

In [8]:
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names, index=all_texts)

In [9]:
for a in tqdm(all_texts):
    with open(f'{a}_23_100.txt', 'a', encoding='utf-8') as f:
        n = pd.Series(df.loc[f'{a}'])
        f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())

In [10]:
del(df)

##### **Эксперимент 2**

In [ ]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

Корпус: 5 документов советских писателей, 5 документов - тексты эмигрантов до эмиграции и 1 документ – тексты всех эмигрантов после эмиграции.

In [29]:
corpus2 = []
emig_after = ''
index2 = []

for b in tqdm(emig):
    with open (f'{b}_после_леммы.txt', encoding='utf-8') as f:
        text = f.read()
    emig_after = ''.join(text)
    with open(f'{b}_до_леммы.txt', encoding='utf-8') as f:
        text = f.read()
        corpus2.append(text)
    index2.append(b)
with open('Эмигранты_после.txt', 'a', encoding='utf-8') as f:
    f.write(emig_after)
    index2.append('Эмигранты')

corpus2.append(emig_after)

for a in tqdm(soviet):
    with open(f'{a}_леммы.txt', encoding='utf-8') as fil:
        text = fil.read()
    corpus2.append(text)
    index2.append(a)

In [30]:
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
print('matrix')
tfidf_matrix = vectorizer.fit_transform(corpus2)
print('features')
feature_names = vectorizer.get_feature_names()
print('dense')
dense = tfidf_matrix.todense()
print('denselist')
denselist = dense.tolist()
print('df')
df2 = pd.DataFrame(denselist, columns=feature_names, index=index2)

matrix
features
dense
denselist
df


In [31]:
with open('Эмигранты_2-3-граммы_после_100.txt', 'a', encoding='utf-8') as f:
    n = pd.Series(df2.loc['Эмигранты'])
    f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())

##### **Эксперимент 3**

In [7]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

5 небольших экспериментов: для каждого эмигранта корпус состоит из 5 документов советских писателей и 1 документа – текстов каждого эмигранта до эмиграции.

In [8]:
def auth_vs_sov(em):
    indexi = soviet
    indexi.append(em)
    corpusi = []
    for a in indexi:
        if a == indexi[-1]:
             with open(f'{a}_до_леммы.txt', encoding='utf-8') as fil:
                    text = fil.read()
        else:
            with open(f'{a}_леммы.txt', encoding='utf-8') as fil:
                text = fil.read()
        corpusi.append(text)
    vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
    tfidf_matrix = vectorizer.fit_transform(corpusi)
    feature_names = vectorizer.get_feature_names()
    dense = tfidf_matrix.todense()
    denselist = dense.tolist()
    dfi = pd.DataFrame(denselist, columns=feature_names, index=indexi)
    with open(f'{em}_до_vs_soviet_23_100.txt', 'a', encoding='utf-8') as f:
        n = pd.Series(dfi.loc[f'{em}'])
        f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())

In [9]:
for em in tqdm(emig):
    auth_vs_sov(em)

##### **Эксперимент 4**

In [10]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

5 небольших экспериментов для каждого автора-эмигранта: корпус состоял из 5 документов советских писателей, 5 документов писателей-эмигрантов до эмиграции и 1 документа с текстами автора-эмигранта, написанными после эмиграции.

In [11]:
def after_vs_before_and_sov(em):
    indexi = []
    corpusi = []
    for a in emig:
        indexi.append(a)
        with open(f'{a}_до_леммы.txt', encoding='utf-8') as fil:
            text = fil.read()
        corpusi.append(text)
    for a in soviet:
        indexi.append(a)
        with open(f'{a}_леммы.txt', encoding='utf-8') as fil:
            text = fil.read()
        corpusi.append(text)
    with open(f'{em}_после_леммы.txt', encoding='utf-8') as fil:
        text = fil.read()
        corpusi.append(text)
        indexi.append(f'{em}_после')
    vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
    tfidf_matrix = vectorizer.fit_transform(corpusi)
    feature_names = vectorizer.get_feature_names()
    dense = tfidf_matrix.todense()
    denselist = dense.tolist()
    dfi = pd.DataFrame(denselist, columns=feature_names, index=indexi)
    with open(f'{em}_vs_before_and_sov_23_100.txt', 'a', encoding='utf-8') as f:
        n = pd.Series(dfi.loc[f'{em}_после'])
        f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())

In [12]:
for em in tqdm(emig):
    after_vs_before_and_sov(em)

##### **Эксперимент 5**

In [17]:
soviet = ['Булгаков', 'Шолохов', 'Фадеев', 'Платонов', 'Стругацкие']
emig = ['Солженицын', 'Бунин', 'Зайцев', 'Ремизов', 'Шмелёв']

Корпус – 5 текстов советских писателей в 1 документе, и 5 документов – текстов эмигрантов после эмиграции. 

In [14]:
index3 = emig
corpus3 = []
for a in tqdm(index3):
    with open(f'{a}_после_леммы.txt', encoding='utf-8') as f:
        text = f.read()
    corpus3.append(text)
index3.append('Советские')
all_sov = ''
for b in soviet:
    with open(f'{b}_леммы.txt', encoding='utf-8') as f:
        text = f.read()
    all_sov = ''.join(text)
corpus3.append(all_sov)

In [15]:
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(corpus3)
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
df3 = pd.DataFrame(denselist, columns=feature_names, index=index3)
with open('soviet_vs_emig_after_23_100.txt', 'a', encoding='utf-8') as f:
    n = pd.Series(df3.loc[f'{index3[-1]}'])
    f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())

Много слов Стругацких, посмотрим без них. Корпус – 4 текста советских писателей в 1 документе, и 5 документов – текстов эмигрантов после эмиграции.

In [18]:
index7 = emig
corpus7 = []
for a in tqdm(index7):
    with open(f'{a}_после_леммы.txt', encoding='utf-8') as f:
        text = f.read()
    corpus7.append(text)
index7.append('Советские')
all_sov_wo_stru = ''
for b in soviet:
    if b != 'Стругацкие':
        with open(f'{b}_леммы.txt', encoding='utf-8') as f:
            text = f.read()
        all_sov_wo_stru = ''.join(text)
corpus7.append(all_sov_wo_stru)

In [19]:
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(corpus7)
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
df7 = pd.DataFrame(denselist, columns=feature_names, index=index7)
with open('soviet_woStrug_vs_emig_after_23_100.txt', 'a', encoding='utf-8') as f:
    n = pd.Series(df7.loc[f'{index7[-1]}'])
    f.write(n[n > 0].sort_values(ascending=False)[:100].to_string())